In [1]:
import math

In [2]:
class Pipe:
    def __init__(self, shape, u, v, distance, prev):
        self.shape = shape
        self.position_u = u # u = column
        self.position_v = v # v = row
        self.distance = distance
        self.next = self._next(prev)

    def _next(self, prev):
        du = self.position_u - prev[0] # column difference
        dv = self.position_v - prev[1] # row difference
        next_u = self.position_u
        next_v = self.position_v
        goto_next = False

        if self.shape == "|":
            next_v += dv
            goto_next = True
        elif self.shape == "-":
            next_u += du
            goto_next = True
        elif self.shape == "L":
            goto_next = True
            if du != 0:
                next_v -= 1
            else:
                next_u += 1
        elif self.shape == "J":
            goto_next = True
            if du != 0:
                next_v -= 1
            else:
                next_u -= 1
        elif self.shape == "7":
            goto_next = True
            if du != 0:
                next_v += 1
            else:
                next_u -= 1
        elif self.shape == "F":
            goto_next = True
            if du != 0:
                next_v += 1
            else:
                next_u += 1
        return [goto_next,next_u, next_v]



In [3]:
pipes_1 = []
pipes_2 = []
data = list()
visited = []
start_pos = []
start_pipe = []
enclosure = list()
with open(r"input-10.txt", "r") as file:
#with open(r"test-input-10.txt", "r") as file:
#with open(r"test-input-10-2.txt", "r") as file:
#with open(r"test-input-10-3.txt", "r") as file:
#with open(r"test-input-10-4.txt", "r") as file:
#with open(r"test-input-10-5.txt", "r") as file:
    for r_num, row in enumerate(file):
        row = row.replace("\n", "")
        split_row = [c for c in list(row)]
        data.append(list(split_row.copy()))
        for i,c in enumerate(split_row):
            if c == "S":
                start_pos = [i, r_num]
                break
    enclosure = [x[:] for x in data] #deep copy
 
    enclosure[start_pos[1]][start_pos[0]] = " "


    aroundS = []
    aroundS.append(enclosure[start_pos[1] - 1][start_pos[0]] if start_pos[1] > 0 else "")
    aroundS.append(enclosure[start_pos[1]][start_pos[0] + 1] if start_pos[0] < (len(enclosure[0]) - 1) else "")
    aroundS.append(enclosure[start_pos[1] + 1][start_pos[0]] if start_pos[1] < (len(enclosure) - 1) else "")
    aroundS.append(enclosure[start_pos[1]][start_pos[0] - 1] if start_pos[0] > 0 else "")

    left = False
    right = False
    top = False
    down = False
    if aroundS[0] in ["|", "7", "F"]:
        start_pipe.append(Pipe(aroundS[0], start_pos[0], start_pos[1] - 1, 1, start_pos))
        top = True
    if aroundS[1] in ["-", "J", "7"]:
        start_pipe.append(Pipe(aroundS[1], start_pos[0] + 1, start_pos[1], 1, start_pos))
        right = True
    if aroundS[2] in ["|", "J", "L"]:
        start_pipe.append(Pipe(aroundS[2], start_pos[0], start_pos[1] + 1, 1, start_pos))
        down = True
    if aroundS[3] in ["-", "L", "F"]:
        start_pipe.append(Pipe(aroundS[3], start_pos[0] - 1, start_pos[1], 1, start_pos))
        left = True

    s = " "
    if left and right:
        s = "-"
    elif right and down:
        s = "F"
    elif top and down:
        s = "|"
    elif top and right:
        s = "L"
    elif top and left:
        s = "J"
    elif left and down:
        s = "7"

    data[start_pos[1]][start_pos[0]] = s

    visited_1 = [[start_pos[1], start_pos[0]], [start_pipe[0].position_v, start_pipe[0].position_u]]
    visited_2 = [[start_pos[1], start_pos[0]], [start_pipe[1].position_v, start_pipe[1].position_u]]
    

    pipes_1.append(start_pipe[0])
    pipes_2.append(start_pipe[1])
    visited.append([start_pipe[0].position_u, start_pipe[0].position_v])
    visited.append([start_pipe[1].position_u, start_pipe[1].position_v])

    middle = False
    it = 1
    while(not middle):
        it += 1
        if [pipes_1[-1].next[1], pipes_1[-1].next[2]] not in visited and pipes_1[-1].next[0] == True:
            next_loc = pipes_1[-1].next
            pipes_1.append(Pipe(enclosure[next_loc[2]][next_loc[1]], next_loc[1], next_loc[2], it, [pipes_1[-1].position_u, pipes_1[-1].position_v]))
            visited.append([next_loc[1], next_loc[2]])
            visited_1.append([next_loc[2], next_loc[1]])
        else:
            middle = True
            break
        if [pipes_2[-1].next[1], pipes_2[-1].next[2]] not in visited and pipes_2[-1].next[0] == True:
            next_loc = pipes_2[-1].next
            pipes_2.append(Pipe(enclosure[next_loc[2]][next_loc[1]], next_loc[1], next_loc[2], it, [pipes_2[-1].position_u, pipes_2[-1].position_v]))
            visited.append([next_loc[1], next_loc[2]])
            visited_2.append([next_loc[2], next_loc[1]])
        else:
            middle = True
            break

    # Region growing of sides
    seeds = []
    num_rows = len(enclosure) 
    # Make data edge points seed points
    for num,i in enumerate(enclosure):
        i[0] = "O"
        i[-1] = "O"
        seeds.append([num, 0])
        seeds.append([num, len(i) - 1])
    enclosure[0][:] = "O" * (len(enclosure[0]))
    enclosure[num_rows - 1][:] = "O" * (len(enclosure[0]))

    for i in range(1, len(enclosure[0])):
        seeds.append([0,i])
        seeds.append([num_rows - 1, i])

    for i in visited:
        enclosure[i[1]][i[0]] = " "
        if [i[1], i[0]] in seeds:
            seeds.remove([i[1], i[0]])
    
    for i, r in enumerate(enclosure):
        for c,e in enumerate(r):
            if enclosure[i][c] != " " and  enclosure[i][c] != "O":
                enclosure[i][c] = "."
    # Grow outside region
    check_next = list(seeds.copy())
    while(check_next != []):
        next_point = check_next.pop(-1)
        if next_point[0] < num_rows - 2: # search below
            if enclosure[next_point[0] + 1][next_point[1]] == ".":
                enclosure[next_point[0] + 1][next_point[1]] = "O"
                check_next.append([next_point[0] + 1, next_point[1]])
        if next_point[1] < len(enclosure[0]) - 2: # search right
            if enclosure[next_point[0]][next_point[1] + 1] == ".":
                enclosure[next_point[0]][next_point[1] + 1] = "O"
                check_next.append([next_point[0], next_point[1] + 1])
        if next_point[1] > 0: # search left
            if enclosure[next_point[0]][next_point[1] - 1] == ".":
                enclosure[next_point[0]][next_point[1] - 1] = "O"
                check_next.append([next_point[0], next_point[1] - 1])
        if next_point[0] > 0: # search above
            if enclosure[next_point[0] - 1][next_point[1]] == ".":
                enclosure[next_point[0] - 1][next_point[1]] = "O"
                check_next.append([next_point[0] - 1, next_point[1]])
    path_list = []
    for i in visited_1:
        path_list.append(i)

    for i in reversed(visited_2):
        path_list.append(i)

    enclosed_coordinates = []
    for i, r in enumerate(enclosure):
        for c,e in enumerate(r):
            if e != "O" and e != " ":
                enclosed_coordinates.append([i, c])
                enclosure[i][c] = "I"

    inliers = [x[:] for x in enclosed_coordinates]
    for i in enclosed_coordinates:
        # Raycast left
        crossings_x = 0
        crossings_ux = 0
        crossings_dx = 0 #differentiate for up and down corners

        for idx in range(i[1] + 1):
            test_r = i[0]
            test_c = i[1] - idx
            if [test_r, test_c] in path_list:
                r_el = data[test_r][test_c]
                if r_el == "-":
                    pass
                elif r_el == "L" or r_el == "J":
                    crossings_ux += 1
                    crossings_x += 1
                elif r_el == "7" or r_el == "F":
                    crossings_dx += 1
                    crossings_x += 1
                else:
                    crossings_x += 1
        if crossings_x % 2 == 0 and crossings_dx % 2 == 0 and crossings_ux % 2 == 0:
            inliers.remove(i)
            enclosure[i[0]][i[1]] = "O"
            check_next.append(i)

        # Raycast right
        crossings_x = 0
        crossings_ux = 0
        crossings_dx = 0

        for idx in range(i[1],len(data[0]) + 1):
            test_r = i[0]
            test_c = idx
            if [test_r, test_c] in path_list:
                r_el = data[test_r][test_c]
                if r_el == "-":
                    pass
                elif r_el == "L" or r_el == "J":
                    crossings_ux += 1
                    crossings_x += 1
                elif r_el == "7" or r_el == "F":
                    crossings_dx += 1
                    crossings_x += 1
                else:
                    crossings_x += 1
        if crossings_x % 2 == 0 and crossings_dx % 2 == 0 and crossings_ux % 2 == 0 and i in inliers:
            inliers.remove(i)
            enclosure[i[0]][i[1]] = "O"
            check_next.append(i)

        # Raycast up
        crossings_y = 0
        crossings_ly = 0
        crossings_ry = 0

        for idx in range(i[0] + 1):
            test_r = i[0] - idx
            test_c = i[1]
            if [test_r, test_c] in path_list:
                r_el = data[test_r][test_c]
                if r_el == "|":
                    pass
                elif r_el == "7" or r_el == "J":
                    crossings_ly += 1
                    crossings_y += 1
                elif r_el == "L" or r_el == "F":
                    crossings_ry += 1
                    crossings_y += 1
                else:
                    crossings_y += 1
        if crossings_y % 2 == 0 and crossings_ly % 2 == 0 and crossings_ry % 2 == 0 and i in inliers:
            inliers.remove(i)
            enclosure[i[0]][i[1]] = "O"
            check_next.append(i)

        # Raycast down
        crossings_y = 0
        crossings_ly = 0
        crossings_ry = 0 
        for idx in range(i[0], num_rows + 1):
            test_r = idx
            test_c = i[1]
            if [test_r, test_c] in path_list:
                r_el = data[test_r][test_c]
                if r_el == "|":
                    pass
                elif r_el == "7" or r_el == "J":
                    crossings_ly += 1
                    crossings_y += 1
                elif r_el == "L" or r_el == "F":
                    crossings_ry += 1
                    crossings_y += 1
                else:
                    crossings_y += 1
        if crossings_y % 2 == 0 and crossings_ly % 2 == 0 and crossings_ry % 2 == 0 and i in inliers:
            inliers.remove(i)
            enclosure[i[0]][i[1]] = "O"
            check_next.append(i)

        # Raycast diagonal up left
        crossings_xy = 0
        min_i = min(i[0],i[1])
        for idx in range(min_i + 1):
            test_r = i[0] - idx
            test_c = i[1] - idx
            if [test_r, test_c] in path_list:
                r_el = data[test_r][test_c]
                if r_el == "7" or r_el == "L":
                    pass
                else:
                    crossings_xy += 1
        if crossings_xy % 2 == 0 and i in inliers:
            inliers.remove(i)
            enclosure[i[0]][i[1]] = "O"
            check_next.append(i)


    # grow inside regions
    while(check_next != []):
        next_point = check_next.pop(-1)
        if next_point[0] < num_rows - 2: # search below
            if enclosure[next_point[0] + 1][next_point[1]] == "I":
                enclosure[next_point[0] + 1][next_point[1]] = "O"
                check_next.append([next_point[0] + 1, next_point[1]])
                inliers.remove([(next_point[0] + 1),next_point[1]])
        if next_point[1] < len(enclosure[0]) - 2: # search right
            if enclosure[next_point[0]][next_point[1] + 1] == "I":
                enclosure[next_point[0]][next_point[1] + 1] = "O"
                check_next.append([next_point[0], next_point[1] + 1])
                inliers.remove([next_point[0],next_point[1] + 1])
        if next_point[1] > 0: # search left
            if enclosure[next_point[0]][next_point[1] - 1] == "I":
                enclosure[next_point[0]][next_point[1] - 1] = "O"
                check_next.append([next_point[0], next_point[1] - 1])
                inliers.remove([next_point[0], next_point[1] - 1])
        if next_point[0] > 0: # search above
            if enclosure[next_point[0] - 1][next_point[1]] == "I":
                enclosure[next_point[0] - 1][next_point[1]] = "O"
                check_next.append([next_point[0] - 1, next_point[1]])
                inliers.remove([next_point[0] - 1, next_point[1]])

print("Part 1:", it)
print("Part 2:",len(inliers))

Part 1: 6846
Part 2: 325
